In [8]:
import boto3
import sagemaker
from sagemaker import get_execution_role
import sys
import IPython



# IPython.Application.instance().kernel.do_shutdown(True)

role = get_execution_role()
sess = sagemaker.Session()
region = boto3.session.Session().region_name
print("Region = {}".format(region))
sm = boto3.Session().client('sagemaker')


Region = ap-south-1


In [9]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os
from time import sleep, gmtime, strftime
import json
import time

In [10]:
#sagemaker experiments to track various training experiments
!pip install sagemaker-experiments

from sagemaker.analytics import ExperimentAnalytics
from smexperiments.experiment import Experiment
from smexperiments.trial import Trial
from smexperiments.trial_component import TrialComponent
from smexperiments.tracker import Tracker

In [12]:
 !wget --header="Host: archive.ics.uci.edu" --header="User-Agent: Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/102.0.0.0 Safari/537.36" --header="Accept: text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9" --header="Accept-Language: en-US,en;q=0.9" --header="Referer: https://archive.ics.uci.edu/ml/machine-learning-databases/00350/" "https://archive.ics.uci.edu/ml/machine-learning-databases/00350/default%20of%20credit%20card%20clients.xls" -c -O 'default of credit card clients.xls'

--2022-06-25 16:47:57--  https://archive.ics.uci.edu/ml/machine-learning-databases/00350/default%20of%20credit%20card%20clients.xls
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5539328 (5.3M) [application/x-httpd-php]
Saving to: ‘default of credit card clients.xls’

default of credit c 100%[===================>]   5.28M  2.42MB/s    in 2.2s    

2022-06-25 16:48:01 (2.42 MB/s) - ‘default of credit card clients.xls’ saved [5539328/5539328]



In [23]:
!pip install sagemaker==2.6.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 291.9/291.9 kB 4.3 MB/s eta 0:00:0000:01
  Preparing metadata (setup.py) ... done
  Created wheel for sagemaker: filename=sagemaker-2.6.0-py2.py3-none-any.whl size=416239 sha256=00a3edc5cda53955f39b24f86102f10f2af0af6497206cef963f06567878f5a8
  Stored in directory: /root/.cache/pip/wheels/27/e1/d4/5b2102915285e40f5448f063effd64a9e405177e65314b866b
Successfully built sagemaker
  Attempting uninstall: smdebug-rulesconfig
    Found existing installation: smdebug-rulesconfig 0.1.4
    Uninstalling smdebug-rulesconfig-0.1.4:
      Successfully uninstalled smdebug-rulesconfig-0.1.4
  Attempting uninstall: sagemaker
    Found existing installation: sagemaker 1.72.0
    Uninstalling sagemaker-1.72.0:
      Successfully uninstalled sagemaker-1.72.0


In [24]:
pd.set_option('display.max_columns',None)

data = pd.read_excel('default of credit card clients.xls', header =1 )


data.rename(columns = {'default payment next month': 'Label'}, inplace = True)
lbl = data.Label
data = pd.concat([lbl, data.drop(columns = ['Label'])], axis = 1)
data.head()

data.drop(columns = ['ID'], axis = 1, inplace = True)


import boto3
import pandas as pd
from sagemaker import get_execution_role

role = get_execution_role()



rawbucket = sess.default_bucket()

prefix = 'sagemaker-modelmonitor'

dataprefix = prefix + '/data'
traindataprefix = prefix + '/train_data'
testdataprefix = prefix + '/test_data'
trainheaderprefix = prefix + '/train_headers'
testdatanolabelprefix = prefix + '/test_data_no_label'

if not os.path.exists('rawdata/rawdata.csv'):
    !mkdir rawdata
    data.to_csv('rawdata/rawdata.csv')
else:
    pass
    




raw_data_location = sess.upload_data('rawdata', bucket = rawbucket, key_prefix = dataprefix)
print(raw_data_location)


from sagemaker.sklearn.processing import SKLearnProcessor
sklearn_processor = SKLearnProcessor( framework_version = '0.20.0',
                                    role = role,
                                  instance_type = 'ml.t3.xlarge',
                                    instance_count = 1)

INFO:sagemaker.image_uris:Same images used for training and inference. Defaulting to image scope: inference.
INFO:sagemaker.image_uris:Defaulting to only available Python version: py3


s3://sagemaker-ap-south-1-333215489353/sagemaker-modelmonitor/data


In [25]:
%%writefile preprocessing.py
import importlib 
import argparse
import os
import warnings
import sklearn
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.exceptions import DataConversionWarning
from sklearn.compose import make_column_transformer

warnings.filterwarnings(action = 'ignore', category = DataConversionWarning)
if __name__ == "__main__":
    parser = argparse.ArgumentParser()
    parser.add_argument('--train-test-split-ratio', type = float, default = 0.3)
    parser.add_argument('--random-split', type = int, default = 0)
    args, _ = parser.parse_known_args()
    
    print('Received arguments {}'.format(args))
    
    input_data_path = os.path.join('/opt/ml/processing/input', 'rawdata.csv')
    print('Reading input data from {}'.format(input_data_path))
    df = pd.read_csv(input_data_path)
    df.sample(frac=1)
    
    COLS = df.columns
    newcolorder = ['PAYAMT1', 'BILL_AMT1'] + list(COLS[1:])[:11] + list(COLS[1:])[12:17]+ list(COLS[1:])[18:]
    
    split_ratio = args.train_test_split_ratio
    random_state = args.random_split
    
    X_train, X_test, y_train, y_test = train_test_split(df.drop('Label', axis = 1), df['Label'],
                                                       test_size = split_ratio, random_state = random_state)
    
    
    preprocess = make_column_transformer(
    (['PAY_AMT1'], StandardScaler()),
    (['BILL_AMT1'], MinMaxScaler()),
    remainder = 'passthrough')
    
    print('Running preprocessing and feature engineering trasnformations')
    train_features = pd.DataFrame(preprocess.fit_transform(X_train), columns = newcolorder)
    test_features = pd.DataFrame(preprocess.fit_transform(X_train), columns = newcolorder)
    
    train_full = pd.concat([pd.DataFrame(y_train.values, columns = ['Label']), train_features], axis = 1)
    test_full = pd.concat([pd.DataFrame(y_test.values, columns = ['Label']), test_features], axis = 1)
    
    print("train data shape after preprocessing : {}".format(train_features.shape))
    print("test data shape after preprocessing : {}".format(test_features.shape))
    
    train_features_headers_output_path = os.path.join('/opt/ml/processing/train_headers', 'train_data_with_headers.csv')
    
    train_features_output_path = os.path.join('/opt/ml/processing/train', 'train_data.csv')
    
    test_features_output_path = os.path.join('/opt/ml/processing/test', 'test_data.csv')
    
    print('Saving training features to {}'.format(train_features_output_path))
    train_full.to_csv(train_features_output_path, header = False, index = False)
    print("Completed")
    
    print('Saving training data with headers to {}'.format(train_features_headers_output_path))
    train_full.to_csv(train_features_headers_output_path, header = False, index = False)
    print("Completed")
    
    print('Saving test features to {}'.format(test_features_output_path))
    test_full.to_csv(test_features_output_path, header = False, index = False)
    print("Completed")
    
    
    

Writing preprocessing.py


In [26]:

# copy the preprocessing code over to the s3 bucket

codeprefix = prefix + '/code'
codeupload= sess.upload_data('preprocessing.py', bucket = rawbucket, key_prefix =codeprefix)
print(codeupload)
# s3 paths to store train and test data after preprocessing
train_data_location = rawbucket + '/' + traindataprefix
test_data_location = rawbucket + '/' + testdataprefix
print("Training data location = {}".format(train_data_location))
print("test data location = {}".format(test_data_location))

s3://sagemaker-ap-south-1-333215489353/sagemaker-modelmonitor/code/preprocessing.py
Training data location = sagemaker-ap-south-1-333215489353/sagemaker-modelmonitor/train_data
test data location = sagemaker-ap-south-1-333215489353/sagemaker-modelmonitor/test_data


In [ ]:
from sagemaker.processing import ProcessingInput, ProcessingOutput

sklearn_processor.run(code = codeupload,
                     inputs = [ProcessingInput(
                     source = raw_data_location,
                     destination = '/opt/ml/processing/input')],
                     outputs = [ProcessingOutput(output_name = 'train_data',
                                                source = '/opt/ml/processing/train',
                                                destination = 's3://' + train_data_location),
                               ProcessingOutput(output_name = 'test_data',
                                                source = '/opt/ml/processing/test',
                                                destination = 's3://' + test_data_location),
                               ProcessingOutput(output_name = 'train_data_headers',
                                                source = '/opt/ml/processing/train_headers',
                                                destination = 's3://' + rawbucket + '/' + prefix + '/train_headers')],
                     arguments = ['--train-test-split-ratio', '0.2']
                     
                     )

preprocessing_job_description = sklearn_processor.jobs[-1].describe()

output_config = preprocessing_job_description['ProcessingOutputConfig']

for output in output_config['Outputs']:
    if output['OutputName'] == 'train_name':
        preprocessed_training_data = output['S3Output']['S3Uri']
    if output['OutputName'] == 'test_data':
        preprocessed_test_data = output['S3Output']['S3Uri']

INFO:sagemaker:Creating processing-job with name sagemaker-scikit-learn-2022-06-25-16-51-59-570



Job Name:  sagemaker-scikit-learn-2022-06-25-16-51-59-570
Inputs:  [{'InputName': 'input-1', 'AppManaged': False, 'S3Input': {'S3Uri': 's3://sagemaker-ap-south-1-333215489353/sagemaker-modelmonitor/data', 'LocalPath': '/opt/ml/processing/input', 'S3DataType': 'S3Prefix', 'S3InputMode': 'File', 'S3DataDistributionType': 'FullyReplicated', 'S3CompressionType': 'None'}}, {'InputName': 'code', 'AppManaged': False, 'S3Input': {'S3Uri': 's3://sagemaker-ap-south-1-333215489353/sagemaker-modelmonitor/code/preprocessing.py', 'LocalPath': '/opt/ml/processing/input/code', 'S3DataType': 'S3Prefix', 'S3InputMode': 'File', 'S3DataDistributionType': 'FullyReplicated', 'S3CompressionType': 'None'}}]
Outputs:  [{'OutputName': 'train_data', 'AppManaged': False, 'S3Output': {'S3Uri': 's3://sagemaker-ap-south-1-333215489353/sagemaker-modelmonitor/train_data', 'LocalPath': '/opt/ml/processing/train', 'S3UploadMode': 'EndOfJob'}}, {'OutputName': 'test_data', 'AppManaged': False, 'S3Output': {'S3Uri': 's3:/

In [29]:
# create a sagemaker experiment

cc_experiment = Experiment.create(
                experiment_name = f"Build-train-deploy-{int(time.time())}",
                description = "Predict credit card default from payments data",
    sagemaker_boto_client = sm)

print(cc_experiment)

Experiment(sagemaker_boto_client=<botocore.client.SageMaker object at 0x7f47c3ccae90>,experiment_name='Build-train-deploy-1656176665',description='Predict credit card default from payments data',tags=None,experiment_arn='arn:aws:sagemaker:ap-south-1:333215489353:experiment/build-train-deploy-1656176665',response_metadata={'RequestId': '0a9b6b91-1293-4459-b1a5-9fedc0b71ad7', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': '0a9b6b91-1293-4459-b1a5-9fedc0b71ad7', 'content-type': 'application/x-amz-json-1.1', 'content-length': '102', 'date': 'Sat, 25 Jun 2022 17:04:25 GMT'}, 'RetryAttempts': 0})


In [30]:
# start tracking parameters used in the pre processing pipeline

with Tracker.create(display_name = "Preprocessing", sagemaker_boto_client = sm) as tracker:
    tracker.log_parameters({
        "train_test_split_ratio":0.2,
        "random_state":0
    })
    
    tracker.log_input(name = "ccdefault-raw-dataset", media_type = "s3/uri", value = raw_data_location)
    tracker.log_input(name = "ccdefault-train-dataset", media_type = "s3/uri", value = train_data_location)
    tracker.log_input(name = "ccdefault-raw-dataset", media_type = "s3/uri", value = train_data_location)
    

In [ ]:
# creating. a trail with xgboost

from sagemaker.amazon.amazon_estimator import get_image_uri
container = get_image_uri(boto3.Session().region_name,'xgboost', '1.0-1') # Docker container to train Xgboost model
s3_input_train = sagemaker.TrainingInput(s3_data = 's3://' + train_data_location, content_type = 'csv')
preprocessing_trial_component = tracker.trial_component

# create trial

trial_name = f"cc-default-training-job-{int(time.time())}"
cc_trial = Trial.create(
              trial_name = trial_name,
              experiment_name = cc_experiment.experiment_name,
              sagemaker_boto_client = sm
          )

cc_trial.add_trial_component(preprocessing_trial_component)
cc_training_job_name = "cc-training-job-{}".format(int(time.time()))

xgb = sagemaker.estimator.Estimator(container,
                                   role,
                                   train_instance_count = 1,
                                    train_instance_type = 'ml.m5.xlarge',
                                    train_max_run = 86400,
                                    output_path = 's3://{}/{}/models'.format(rawbucket, prefix),
                                    sagemaker_session = sess)


xgb.set_hyperparameters(max_depth = 5,
                       eta = 0.2,
                       gamma = 4,
                       min_child_weight = 6,
                       subsample = 0.8,
                       verbosity = 0,
                       objective = 'binary:logistic',
                       num_round = 100)

xgb.fit(inputs = {'train':s3_input_train},
       job_name = cc_training_job_name,
       experiment_config = {
           "TrialName":cc_trial.trial_name,
           "TrialComponentDisplayName":"Training",
           
       },
       wait = True,)


time.sleep(2)
                                    

See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
INFO:sagemaker.image_uris:Same images used for training and inference. Defaulting to image scope: inference.
INFO:sagemaker.image_uris:Defaulting to only available Python version: py3
INFO:sagemaker.image_uris:Defaulting to only supported image scope: cpu.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
INFO:sagemaker.image_uris:Defaulting to the only supported framework/algorithm version: latest.
INFO:sagemaker.image_uris:Ignoring unnecessary instance type: None.
INFO:sagemaker:Creating training-job with name: cc-training-job-1656179704


2022-06-25 17:55:04 Starting - Starting the training job...
2022-06-25 17:55:20 Starting - Preparing the instances for trainingProfilerReport-1656179704: InProgress
......
2022-06-25 17:56:21 Downloading - Downloading input data...
2022-06-25 17:56:58 Training - Downloading the training image...Training seconds: 93
Billable seconds: 93


In [40]:
test_data_path = 's3://' + test_data_location + '/test_data.csv'
! aws s3 cp $test_data_path .

download: s3://sagemaker-ap-south-1-333215489353/sagemaker-modelmonitor/test_data/test_data.csv to ./test_data.csv


In [42]:
test_full = pd.read_csv('test_data.csv', names = [str(x) for x in range(len(data.columns))])
test_full.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23
0.0,-0.160932,0.200215,3225.0,20000.0,2.0,3.0,1.0,44.0,0.0,0.0,2.0,0.0,0.0,-2.0,19112.0,17980.0,18780.0,0.0,0.0,0.0,1000.0,1000.0,0.0,0.0
0.0,-0.331546,0.181659,11815.0,260000.0,2.0,2.0,2.0,30.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,165.0,274.0,165.0,333.0,165.0,274.0,165.0,333.0,165.0,293.0
0.0,-0.251204,0.203688,7338.0,20000.0,1.0,2.0,1.0,39.0,2.0,0.0,0.0,0.0,0.0,0.0,20328.0,19299.0,19928.0,20204.0,20398.0,1500.0,900.0,700.0,1480.0,0.0
0.0,-0.233149,0.212413,14980.0,30000.0,1.0,2.0,1.0,23.0,2.0,2.0,2.0,2.0,2.0,2.0,29276.0,28635.0,30127.0,30525.0,29793.0,150.0,2250.0,1000.0,0.0,700.0
0.0,-0.172968,0.190548,27167.0,10000.0,1.0,2.0,1.0,29.0,0.0,0.0,0.0,0.0,0.0,0.0,8409.0,8600.0,9470.0,6690.0,9690.0,2000.0,1500.0,900.0,3000.0,0.0


In [44]:
label = test_full['0']

In [ ]:
%%timeit

sm_transformer = xgb.transformer(1, 'ml.m5.xlarge', accept = 'text/csv')
sm_transformer.transform(test_data_path, split_type = 'Line', input_filter = '$[1:]', content_type = 'text/csv')
sm_transformer.wait(1)

INFO:sagemaker:Creating model with name: sagemaker-xgboost-2022-06-26-09-27-42-229
INFO:sagemaker:Creating transform job with name: sagemaker-xgboost-2022-06-26-09-27-42-643


........................
[2022-06-26:09:31:28:INFO] No GPUs detected (normal if no gpus installed)
[2022-06-26:09:31:28:INFO] No GPUs detected (normal if no gpus installed)
[2022-06-26:09:31:28:INFO] nginx config: 
worker_processes auto;
daemon off;
pid /tmp/nginx.pid;
error_log  /dev/stderr;
worker_rlimit_nofile 4096;
events {
  worker_connections 2048;
}
http {
  include /etc/nginx/mime.types;
  default_type application/octet-stream;
  access_log /dev/stdout combined;
  upstream gunicorn {
    server unix:/tmp/gunicorn.sock;
  }
  server {
    listen 8080 deferred;
    client_max_body_size 0;
    keepalive_timeout 3;
    location ~ ^/(ping|invocations|execution-parameters) {
      proxy_set_header X-Forwarded-For $proxy_add_x_forwarded_for;
      proxy_set_header Host $http_host;
      proxy_redirect off;
      proxy_read_timeout 60s;
      proxy_pass http://gunicorn;
    }
    location / {
      return 404 "{}";
    }
  }
}
[2022-06-26 09:31:28 +0000] [19] [INFO] Starting gunicorn 1

INFO:sagemaker:Creating model with name: sagemaker-xgboost-2022-06-26-09-31-55-782
INFO:sagemaker:Creating transform job with name: sagemaker-xgboost-2022-06-26-09-31-56-110


........................
[2022-06-26:09:35:44:INFO] No GPUs detected (normal if no gpus installed)
[2022-06-26:09:35:44:INFO] No GPUs detected (normal if no gpus installed)
[2022-06-26:09:35:44:INFO] No GPUs detected (normal if no gpus installed)
[2022-06-26:09:35:44:INFO] No GPUs detected (normal if no gpus installed)
[2022-06-26:09:35:44:INFO] nginx config: 
worker_processes auto;
daemon off;
pid /tmp/nginx.pid;
error_log  /dev/stderr;
worker_rlimit_nofile 4096;
events {
  worker_connections 2048;
}
http {
  include /etc/nginx/mime.types;
  default_type application/octet-stream;
  access_log /dev/stdout combined;
  upstream gunicorn {
    server unix:/tmp/gunicorn.sock;
  }
  server {
    listen 8080 deferred;
    client_max_body_size 0;
    keepalive_timeout 3;
    location ~ ^/(ping|invocations|execution-parameters) {
      proxy_set_header X-Forwarded-For $proxy_add_x_forwarded_for;
      proxy_set_header Host $http_host;
      proxy_redirect off;
      proxy_read_timeout 60s;
   

INFO:sagemaker:Creating model with name: sagemaker-xgboost-2022-06-26-09-36-09-194


...

.........................[2022-06-26:09:52:45:INFO] No GPUs detected (normal if no gpus installed)
[2022-06-26:09:52:45:INFO] No GPUs detected (normal if no gpus installed)
[2022-06-26:09:52:45:INFO] nginx config: 
worker_processes auto;
daemon off;
pid /tmp/nginx.pid;
error_log  /dev/stderr;
worker_rlimit_nofile 4096;
events {
  worker_connections 2048;
}
[2022-06-26:09:52:45:INFO] No GPUs detected (normal if no gpus installed)
[2022-06-26:09:52:45:INFO] No GPUs detected (normal if no gpus installed)
[2022-06-26:09:52:45:INFO] nginx config: 
worker_processes auto;
daemon off;
pid /tmp/nginx.pid;
error_log  /dev/stderr;
worker_rlimit_nofile 4096;
events {
  worker_connections 2048;
}
http {
  include /etc/nginx/mime.types;
  default_type application/octet-stream;
  access_log /dev/stdout combined;
  upstream gunicorn {
    server unix:/tmp/gunicorn.sock;
  }
  server {
    listen 8080 deferred;
    client_max_body_size 0;
    keepalive_timeout 3;
    location ~ ^/(ping|invocations|exec

INFO:sagemaker:Creating model with name: sagemaker-xgboost-2022-06-26-09-53-31-573


[2022-06-26:09:52:45:INFO] No GPUs detected (normal if no gpus installed)
[2022-06-26:09:52:45:INFO] No GPUs detected (normal if no gpus installed)
[2022-06-26:09:52:45:INFO] nginx config: 
worker_processes auto;
daemon off;
pid /tmp/nginx.pid;
error_log  /dev/stderr;
worker_rlimit_nofile 4096;
events {
  worker_connections 2048;
}
[2022-06-26:09:52:45:INFO] No GPUs detected (normal if no gpus installed)
[2022-06-26:09:52:45:INFO] No GPUs detected (normal if no gpus installed)
[2022-06-26:09:52:45:INFO] nginx config: 
worker_processes auto;
daemon off;
pid /tmp/nginx.pid;
error_log  /dev/stderr;
worker_rlimit_nofile 4096;
events {
  worker_connections 2048;
}
http {
  include /etc/nginx/mime.types;
  default_type application/octet-stream;
  access_log /dev/stdout combined;
  upstream gunicorn {
    server unix:/tmp/gunicorn.sock;
  }
  server {
    listen 8080 deferred;
    client_max_body_size 0;
    keepalive_timeout 3;
    location ~ ^/(ping|invocations|execution-parameters) {
     

INFO:sagemaker:Creating transform job with name: sagemaker-xgboost-2022-06-26-09-53-32-152


....................

In [ ]:
import json
import io
from urllib.parse import urlparse

def get_csv_output_from_s3(s3uri, file_name):
    parse_url = urlparse(s3uri)
    bucket_name = parsed_url.netloc
    prefix = parsed_url.path[1:]
    s3 = boto3.resource('s3')
    obj = s3.Object(bucket_name, '{}/{}'.format(prefix, file_name))
    return obj.get()["Body"].read().decode('utf-8')

output = get_csv_output_from_s3(sm_transformer.output_path, 'test_data.csv.out')
output_df = pd.read_csv(io.StringIO(output), sep = ",", header = None)
output_df.head(8)

output_df['Predicted'] = np.round(output_df.values)
output_df['Label'] = label
from sklearn.metrics import confusion_matrix, accuracy_score
confusion_matrix = pd.crosstab(output_df['Predicted'], output_df['Label'], rownames = ['Actual'], colnames = ['Predicted'],
                              margins = True)
confusion_matrix